# Compilation using a locally created calibration set

This notebook shows how to construct a calibration set locally and use the local calibration set for compiling circuits.

The calibration itself is done using IQM's Graph Based Calibration (GBC) software. In this notebook we show recalibration based on the default calibration set. The usage of Pulla with a local calibration set works mostly the same way also for a full calibration from scratch. In that case you would just have to use a different GBC graph, and possibly manually add observations to the observation stash if it does not yet have all observations required for a valid calibration set.

Using the principles shown in this notebook, it is possible to use all Pulla functionality as a helpful tool during the calibration process.

**NOTE:** Some of the libraries used in this notebook are only available to on-premise customers and are not currently available publicly.

In [ ]:
import os

from calibration_graphs import CrystalRecalibrationGraphBuilder
from exa.core.calibration_set_wrapper import CalibrationSetWrapper
from exa.core.provider import provider as exa_provider
import gbc
from qiskit import QuantumCircuit, visualization
from qiskit.compiler import transpile

from iqm.cpc.compiler.compiler import Compiler
from iqm.pulla.pulla import Pulla
from iqm.pulla.utils import calset_from_observations
from iqm.pulla.utils_qiskit import qiskit_to_pulla, sweep_job_to_qiskit
from iqm.qiskit_iqm import IQMProvider
from iqm.qiskit_iqm.iqm_transpilation import optimize_single_qubit_gates

First we prepare a Pulla object, a `qiskit_iqm` backend, and a circuit as explained in `Quick Start.ipynb`.

In [ ]:
iqm_server_url = os.environ['PULLA_IQM_SERVER_URL']  # or set the URL directly here
os.environ["IQM_TOKEN"] = os.environ.get("IQM_TOKEN")  # or set the token directly here

p = Pulla(iqm_server_url)
provider = IQMProvider(iqm_server_url)
backend = provider.get_backend()

qc = QuantumCircuit(3, 3)
qc.h(0)
qc.cx(0, 1)
qc.cx(0, 2)
qc.measure_all()

qc_transpiled = transpile(qc, backend=backend, layout_method='sabre', optimization_level=3)
qc_optimized = optimize_single_qubit_gates(qc_transpiled)
circuits, compiler = qiskit_to_pulla(p, backend, qc_optimized)

Let's also define a function that compiles and runs the above circuit using the given compiler state. This function will be used to compile and execute the circuit at different points of the calibration process:

In [ ]:
def compile_and_execute_circuit(c: Compiler):
    shots = 1000
    playlist, context = c.compile(circuits)
    settings, context = c.build_settings(context, shots=shots)

    print(f"Playlist:\n{playlist}")
    print(f"Settings:")
    settings.print_tree()

    job = p.submit_playlist(playlist, settings, context=context)
    job.wait_for_completion()
    qiskit_result = sweep_job_to_qiskit(job, shots=shots, execution_options=context['options'])

    print(f"Qiskit result counts:\n{qiskit_result.get_counts()}\n")
    visualization.plot_histogram(qiskit_result.get_counts())


First we compile and execute the circuit using the compiler obtained above, which uses the current default calibration set on the server. The results of this execution can then be compared to the results after performing (parts of) the recalibration process.

In [4]:
compile_and_execute_circuit(compiler)

Playlist:
Schedule info:
 - 6 channels
 - 1 segments
 - 9 unique waveforms
 - 29 unique instructions

 QB3__drive.awg:
    Instruction(duration_samples=598000, operation=Wait())
    Instruction(duration_samples=80, operation=IQPulse(wave_i=TruncatedGaussian(n_samples=80, full_width=1.4999999999999998, center_offset=0.0), wave_q=TruncatedGaussianDerivative(n_samples=80, full_width=1.4999999999999998, center_offset=0.0), scale_i=0.0951287465369903, scale_q=-0.0034765998160462136, phase=-1.5707963267948966, modulation_frequency=0.0, phase_increment=0))
    Instruction(duration_samples=192, operation=VirtualRZ(phase_increment=-3.5601317605991643))
    Instruction(duration_samples=152, operation=VirtualRZ(phase_increment=-3.6573362424802793))
    Instruction(duration_samples=3624, operation=Wait())

    TruncatedGaussian(n_samples=80, full_width=1.4999999999999998, center_offset=0.0)
    TruncatedGaussianDerivative(n_samples=80, full_width=1.4999999999999998, center_offset=0.0)

 QB1__drive

Now we will start recalibration using GBC. We need a CalibrationSetWrapper instance for handling calibration sets and a Calibrator for running the calibration:

In [ ]:
# CalibrationSetWrapper requires explicitly initializing station control and experiment configuration.
station_control_url = os.environ['PULLA_STATION_CONTROL_URL']  # or set the URL directly here
exa_provider.init_station_control(station_control_url, get_token_callback=p._iqm_server_client._token_callback)
exa_provider.init_experiment_configuration("experiment.yml")
# Obtain CalibrationSetWrapper and observation stash based on the default calibration set.
chip_topology = p.get_chip_topology()
qubits_to_calibrate = ['QB1', 'QB2', 'QB3', 'QB5']  # Set here the qubits you want to calibrate
wrapper = CalibrationSetWrapper(
    qubits_to_calibrate,
    [],  # no computational resonators
    chip_topology.get_connecting_couplers(qubits_to_calibrate),
    chip_topology.get_connected_probe_lines(qubits_to_calibrate)
)
stash_before_recalibration = wrapper.observation_stash_from_calset(p.fetch_default_calibration_set()[1])

# Initialize the GBC calibration graph and calibrator.
# NOTE: This is a conceptual example that may not work with the latest GBC version. See GBC docs for up-to-date instructions.
graph = CrystalRecalibrationGraphBuilder(
    qubits=qubits_to_calibrate,
    include_benchmarks_rb=True,
    include_benchmarks_coherence=True,
    include_cz_recalibration=True,
).build()
calibrator = gbc.Calibrator(graph)
calibrator.initial_stash = stash_before_recalibration.get_latest_observations().values()  # recalibration will start from observations of default calset obtained above

To start with, we run the first node that runs any actual calibration experiments:

In [12]:
calibrator.calibrate(calibrator.nodes.freq_initial_recal)

Calibrating node: set_default_implementations
set_default_implementations_#0 completed in 1.0036234855651855 secs -> SUCCESS Ⓒ-1.0
Calibrating node: set_target_freq_recal
set_target_freq_recal_#0 completed in 1.6071202754974365 secs -> SUCCESS Ⓒ1.0
Calibrating node: freq_initial_recal
[01-24 23:00:23;I] Celery task ID: 612e95cf-3943-4de6-a141-03eef7f12cb7
freq_initial_recal_#0 completed in 7.035228490829468 secs -> SUCCESS Ⓒ0.9742555921904078
Calibration process Recalibration graph succeeded


Calibration finished.

Then we can create a local calibration set based on the results of the above calibration node, and update the compiler to use this calibration set. We compile and execute a circuit using the updated compiler to see how our updates to the calibration set affect the results. Because many observations of the updated calibration set don't yet take into account the results of the executed node, the results could even be worse than before updating the calibration set with these intermediate results.

In [13]:
# Obtain observation stash based on results of the given calibration node and its dependencies.
stash_after_calibration_node = calibrator.get_observations(calibrator.nodes.freq_initial_recal)
# Use the calset wrapper to obtain only the observations that constitute a calibration set.
observations = wrapper.filter_stash(stash_after_calibration_node)
# Convert into Pulla calibration set format.
calset = calset_from_observations(observations)

# Update the compiler to use the local calibration set and compile+execute a circuit using it.
compiler.set_calibration_set_values(calset)
compile_and_execute_circuit(compiler)

Playlist:
Schedule info:
 - 6 channels
 - 1 segments
 - 9 unique waveforms
 - 29 unique instructions

 QB3__drive.awg:
    Instruction(duration_samples=598000, operation=Wait())
    Instruction(duration_samples=80, operation=IQPulse(wave_i=TruncatedGaussian(n_samples=80, full_width=1.4999999999999998, center_offset=0.0), wave_q=TruncatedGaussianDerivative(n_samples=80, full_width=1.4999999999999998, center_offset=0.0), scale_i=0.0951287465369903, scale_q=-0.0034765998160462136, phase=-1.5707963267948966, modulation_frequency=0.0, phase_increment=0))
    Instruction(duration_samples=192, operation=VirtualRZ(phase_increment=-3.5601317605991643))
    Instruction(duration_samples=152, operation=VirtualRZ(phase_increment=-3.6573362424802793))
    Instruction(duration_samples=3624, operation=Wait())

    TruncatedGaussian(n_samples=80, full_width=1.4999999999999998, center_offset=0.0)
    TruncatedGaussianDerivative(n_samples=80, full_width=1.4999999999999998, center_offset=0.0)

 QB1__drive

We could repeat the above steps multiple times for different nodes of the calibration graph, to see how the results change during the calibration process.

Finally, we run the full recalibration, which runs all the remaining nodes that we did not run above:

In [14]:
calibrator.calibrate(calibrate_all=True)

Calibrating node: set_default_implementations
Calibrating node: set_target_freq_recal
Calibrating node: freq_initial_recal
Calibrating node: readout_opt_recal
[01-24 23:00:53;I] Celery task ID: 6b7bca61-6252-4fcc-a719-301268b2451f
readout_opt_recal_#0 completed in 6.349461793899536 secs -> SUCCESS Ⓒ0.90675
Calibrating node: drag_gaussian_iterative_parking_recal_H1
[01-24 23:01:00;I] Celery task ID: d7d3a614-a159-42ae-bdd2-0ed17cd99e7c
All qubits ready. Stop iteration.
drag_gaussian_iterative_parking_recal_H1_#0 completed in 21.51668119430542 secs -> SUCCESS Ⓒ1.0
Calibrating node: drag_gaussian_iterative_parking_recal_H2
[01-24 23:01:21;I] Celery task ID: 88a30ec7-74a4-4eeb-aa9c-ed72a65177c1
All qubits ready. Stop iteration.
drag_gaussian_iterative_parking_recal_H2_#0 completed in 21.33298945426941 secs -> SUCCESS Ⓒ1.0
Calibrating node: drag_gaussian_qbangbang_recal_H1
[01-24 23:01:42;I] Celery task ID: afaff90c-cb98-4485-9404-517e9a9fb8de
drag_gaussian_qbangbang_recal_H1_#0 completed i

Iterations of the circuit:   0%|          | 0/5 [00:00<?, ?it/s]

T1_recal_H1_#0 completed in 64.67484092712402 secs -> SUCCESS Ⓒ1.0
Calibrating node: T2Ramsey_recal_H1


[01-24 23:17:49;W] The drive frequency of QB4 is set to None, can't detune it.


[01-24 23:17:50;I] Celery task ID: 229a3a18-0478-4912-b062-2edd90bd6292


...

tgss_cz_rb_nn_group_recal_S3_#0 completed in 117.58515524864197 secs -> SUCCESS Ⓒ0.9774642638806206
Calibrating node: full_run_with_benchmarks_recal


100%|██████████| 3/3 [00:00<00:00, 1808.15it/s]


full_run_with_benchmarks_recal_#0 completed in 1.968658208847046 secs -> SUCCESS Ⓒ0.9806514316988835
Calibrating node: full_recalibration
full_recalibration_#0 completed in 0.8926033973693848 secs -> SUCCESS Ⓒ-1.0
Calibrating node: calibration_set
calibration_set_#0 completed in 0.9779641628265381 secs -> SUCCESS Ⓒ-1.0
Calibrating node: quality_metric_set
quality_metric_set_#0 completed in 0.9855170249938965 secs -> SUCCESS Ⓒ1
Calibration process Recalibration graph succeeded


Calibration finished.

<Figure size 640x480 with 0 Axes>

Then we again update the compiler using the results of the calibration in the same way as before. If the full recalibration was successful, the execution results should hopefully be at least as good as before the recalibration.

In [15]:
stash_after_full_recalibration = calibrator.get_observations(calibrator.nodes.full_recalibration)  # now we use observations from the full recalibration
observations = wrapper.filter_stash(stash_after_full_recalibration)
calset = calset_from_observations(observations)

compiler.set_calibration_set_values(calset)
compile_and_execute_circuit(compiler)

Playlist:
Schedule info:
 - 6 channels
 - 1 segments
 - 9 unique waveforms
 - 29 unique instructions

 QB3__drive.awg:
    Instruction(duration_samples=598000, operation=Wait())
    Instruction(duration_samples=80, operation=IQPulse(wave_i=TruncatedGaussian(n_samples=80, full_width=1.4999999999999998, center_offset=0.0), wave_q=TruncatedGaussianDerivative(n_samples=80, full_width=1.4999999999999998, center_offset=0.0), scale_i=0.09510904221394137, scale_q=-0.0034453074886052846, phase=-1.5707963267948966, modulation_frequency=0.0, phase_increment=0))
    Instruction(duration_samples=184, operation=VirtualRZ(phase_increment=-3.5585511910175773))
    Instruction(duration_samples=152, operation=VirtualRZ(phase_increment=-3.6347383064376295))
    Instruction(duration_samples=3624, operation=Wait())

    TruncatedGaussian(n_samples=80, full_width=1.4999999999999998, center_offset=0.0)
    TruncatedGaussianDerivative(n_samples=80, full_width=1.4999999999999998, center_offset=0.0)

 QB1__driv